In [ ]:
# Function to decode operation

In [1]:
%pylab inline 

import mne
from mne.datasets import spm_face
from mne.decoding import GeneralizationAcrossTime
import sys
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
from scipy import stats
#Add personal functions to python path
sys.path.append('/neurospin/meg/meg_tmp/Calculation_Pedro_2014/scripts/decoding/')
#sys.path.append('/Volumes/NeuroSpin2T/Calculation_Pedro_2014/scripts/decoding/')
from fldtrp2mne import fldtrp2mne
from calc_ClassifyTwoCond import calc_ClassifyTwoCond
from sklearn import svm
from sklearn.cross_validation import cross_val_score, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import StratifiedKFold

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Directories
data_path = '/neurospin/meg/meg_tmp/Calculation_Pedro_2014/data/mat/'
result_path = '/neurospin/meg/meg_tmp/Calculation_Pedro_2014/data/decoding/'
#data_path = '/Volumes/NeuroSpin2T/Calculation_Pedro_2014/data/mat/'
#result_path = '/Volumes/NeuroSpin2T/Calculation_Pedro_2014/data/decoding/'

#Subjects
#subjects = ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10', 
            #'s11', 's12', 's13', 's14', 's15', 's16', 's17', 's18','s19', 's21', 's22']

subjects = ['s01']


#General parameters
baseline = (-0.5, -0.05)

downsmpDec = 100

#Decoding
trainset = 'op1'
testset = 'prestResultCorrect'
decCond = ['3', '4', '5', '6']

params = {'baseline': baseline, 'downsmpDec': downsmpDec, 
'Classification': decCond, 'trainset': trainset, 'testset': testset}

#Results initialization
all_scores = []
all_diagonals = []

In [ ]:
for sub in subjects:
    fname = op.join(data_path, sub + '_calc.mat') 
    epoch = fldtrp2mne(fname, 'data')

    #Baseline-correct & filter data
    print('Baseline-correcting data for subject: ' + sub)
    epoch.apply_baseline(baseline)
    
    #Load condition/behavior info
    matfile = sio.loadmat(fname)
    print('Loading trialinfo for subject: ' + sub)   
    trialinfo = matfile['data']['trialinfo']   
    
    run = trialinfo[0][0][0][0][0].T
    operand1 = trialinfo[0][0][0][0][1].T
    operator = trialinfo[0][0][0][0][2].T
    operand2 = trialinfo[0][0][0][0][3].T
    preResult = trialinfo[0][0][0][0][4].T
    delay = trialinfo[0][0][0][0][5].T
    corrResult = trialinfo[0][0][0][0][6].T
    deviant = trialinfo[0][0][0][0][7].T
    absdeviant = trialinfo[0][0][0][0][8].T
    rt = trialinfo[0][0][0][0][9].T
    respSide = trialinfo[0][0][0][0][10].T

    
    # Merge back and define conditions
    info = pd.DataFrame(data = np.concatenate((run, operand1, operator, operand2, preResult, delay, corrResult, deviant, absdeviant, rt, respSide), axis = 1), 
                        columns = ['run', 'operand1', 'operator', 'operand2', 'preResult', 'delay', 'corrResult', 'deviant', 'absdeviant', 'rt', 'respSide'])

    #info['absdeviant'][info['absdeviant'] != 0] = 1
    op13456 = (info['operand1'] >= 3) & (info['operand1'] <= 6)  
    res3456 = (info['corrResult'] >= 3) & (info['corrResult'] <= 6) & (info['operator'] != 0) 
    
    # Make train and test same size
    #op13456 = op13456[0:len(res3456)]
    
    
    # Decoding
    epochs_op1 = epoch[op13456] #
    info_op1 = info[op13456]    #
    y_op1 = np.array(info_op1['operand1']) #
    y_op1 = y_op1.astype(numpy.float64)
    # Downsampling for decoding
    epochs_op1.decimate(downsmpDec)
    
    # Decoding
    epochs_res = epoch[res3456] #
    info_res = info[res3456]    #
    y_res = np.array(info_res['corrResult']) #
    y_res = y_res.astype(numpy.float64)
    # Downsampling for decoding
    epochs_res.decimate(downsmpDec)

    print('Decoding subject: ' + sub)
    gat, score, diagonal = calc_ClassifyTwoCond(epochs_op1, y_op1, epochs_res, y_res, params)
    gat.plot()
    gat.plot_diagonal()  # plot decoding across time (correspond to GAT diagonal)

    #Store scores of different subjects in the same list
    all_scores.append(score)
    all_diagonals.append(diagonal)
    
#Transform into a numpy array   
all_scores = np.array(all_scores)
all_diagonals = np.array(all_diagonals)

# Save individual results
fname = op.join(result_path, 'Classification_ ' + decCond[0] + '_vs_' + 
                decCond[1] + '_vs_' + decCond[2] + '_vs_' + decCond[3] + '_Trainset_' + trainset + '_Testset_' + testset) 
np.save(fname, all_scores)

# Compute group averages
group_scores = np.mean(all_scores, 0)
sem_group_scores = stats.sem(all_scores, 0)

group_diagonal = np.mean(all_diagonals, 0)
sem_group_diagonal = stats.sem(all_diagonals, 0)
    

Baseline-correcting data for subject: s01
Applying baseline correction ... (mode: mean)
Loading trialinfo for subject: s01
Decoding subject: s01


/volatile/anaconda/lib/python2.7/site-packages/mne/epochs.py:394: UserWarning: The measurement information indicates a low-pass frequency of 125 Hz. The decim=4 parameter will result in a sampling frequency of 62.5 Hz, which can cause aliasing artifacts.
  % (lowpass, decim, new_sfreq))  # > 50% nyquist limit


In [ ]:
epochs_op1

In [ ]:
epochs_res

In [ ]:
# Plotting  

# Plot GAT
plt.imshow(group_scores, origin = 'lower', extent = [epochs.times[0], epochs.times[len(epochs.times)-1], 
                                                     epochs.times[0], epochs.times[len(epochs.times)-1]]) #flip the matrix around
plt.axvline(0, color = 'k') #mark stimulus onset
plt.axhline(0, color = 'k') #mark stimulus onset
plt.colorbar()
plt.xlabel('Testing Time (s)')
plt.ylabel('Training Time (s)')
plt.title('Group average generalization across time \n Classification: ' + decCond[0] + ' vs ' 
+ decCond[1] + '\n Trainset: ' + trainset + ', Testset: ' + testset)



In [ ]:
# Plot Diagonal
plt.plot(epochs.times, group_diagonal, label = "Classification score")
plt.axvline(0, color = 'k') #mark stimulus onset
plt.axvline(.8, color = 'k') #mark stimulus onset
plt.axvline(1.6, color = 'k') #mark stimulus onset
plt.axvline(2.4, color = 'k') #mark stimulus onset
plt.axvline(3.2, color = 'k') #mark stimulus onset
plt.axvline(3.6, color = 'k') #mark stimulus onset

plt.axhline(0.25, color = 'k', ls = '--', label = "Chance") #mark chance level
plt.legend(loc = 'upper right')
plt.xlabel('Time (s)')
plt.ylabel('Classification Score (%)')
plt.title('Cl assification: ' + decCond[0] + ' vs ' 
+ decCond[1] + '\n Trainset: ' + trainset + ', Testset: ' + testset)


In [ ]:
plt.imshow(group_scores[10:100, 10:100])
plt.colorbar()


In [ ]:
a = 'a'

In [ ]:
a == 'a'